In [1]:
import sys
sys.path.append("../bert")
sys.path.append("../..")
import multiprocessing as mp
from typing import Counter
import argparse
import logging
import time
from tqdm import tqdm
from data_structure.question import Question, NewQuestion
import pandas as pd
from util.util import get_files_paths_from_directory


In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
        "microsoft/codebert-base", local_files_only=True)
def gen_feature(tokens, max_length):

        feature = tokenizer(tokens, max_length=max_length,
                                 padding='max_length', return_attention_mask=True,
                                 return_token_type_ids=False, truncation=True,
                                 return_tensors='pt')
        res = {
            "input_ids": feature["input_ids"].flatten(),
            "attention_mask": feature["attention_mask"].flatten()}
        return res

In [3]:
def process_file_to_tensor(file):
    out_dir = "../data/tensor_data/"
    dataset = pd.read_pickle(file)
    file_name = file[24:]
    q_list = list()
    for question in dataset:
        qid = question.get_qid()
        title = question.get_title()
        title_feature = gen_feature(title, 100)
        text = question.get_text()
        text_feature = gen_feature(text, 512)
        code = question.get_code()
        code_feature = gen_feature(code, 512)
        date = question.get_creation_date()
        tags = question.get_tag()
        q_list.append(NewQuestion(qid, title_feature, text_feature, code_feature, date, tags))
    import pickle
    with open(out_dir+file_name, 'wb') as f:
        pickle.dump(q_list, f)

In [4]:
from tqdm import tqdm
import time
import multiprocessing as mp
files = get_files_paths_from_directory("../data/processed_train")

pbar = tqdm(total=len(files))
update = lambda *args: pbar.update()
start_time = time.time()
pool = mp.Pool(mp.cpu_count())
for file in files:
    print(file)
    pool.apply_async(process_file_to_tensor, args=(file, ), callback=update)
    # result.get()
pool.close()
pool.join()

  0%|          | 0/514 [00:00<?, ?it/s]

../data/processed_train/train-0-20000.pkl
../data/processed_train/train-100000-120000.pkl
../data/processed_train/train-1000000-1020000.pkl
../data/processed_train/train-10000000-10020000.pkl
../data/processed_train/train-10020000-10040000.pkl
../data/processed_train/train-10040000-10060000.pkl
../data/processed_train/train-10060000-10080000.pkl
../data/processed_train/train-10080000-10100000.pkl
../data/processed_train/train-10100000-10120000.pkl
../data/processed_train/train-10120000-10140000.pkl
../data/processed_train/train-10140000-10160000.pkl
../data/processed_train/train-10160000-10180000.pkl
../data/processed_train/train-10180000-10200000.pkl
../data/processed_train/train-1020000-1040000.pkl
../data/processed_train/train-10200000-10220000.pkl
../data/processed_train/train-10220000-10240000.pkl
../data/processed_train/train-10240000-10260000.pkl
../data/processed_train/train-10260000-10279014.pkl
../data/processed_train/train-1040000-1060000.pkl
../data/processed_train/train-10

100%|██████████| 514/514 [18:15<00:00,  2.14s/it]

In [5]:
files = get_files_paths_from_directory("../data/tensor_data")


In [8]:
file = files[0]
dataset = pd.read_pickle(file)

In [13]:
type(dataset[0].get_text()["input_ids"])

torch.Tensor